# Data aggregation of prepared data

## Utils

In [3]:
#hide
import os
import inspect
import pandas as pd
import json
import time
import datetime
from datetime import timedelta
import warnings
import logging
from functools import reduce

from functools import wraps
import datetime as dt

from dateutil.relativedelta import relativedelta
from copy import deepcopy

logging.basicConfig(level=logging.INFO)
warnings.filterwarnings('ignore')

The following versions of libraries have been used :

In [4]:
print('Pandas :',pd.__version__)

Pandas : 1.3.4


In [5]:
#IMPORTANT: where to find the data? 
# inside the project data-sandbox-fr on GCP, inside Google Cloud Storage
# in the bucket m5-forecast/camille-uncertainty-with-catboost
# you can use the folder prepared_data

df = pd.read_pickle('prepared_data/M5_df.pkl')

In [6]:
df_macro = df.drop(columns=['snap_CA','snap_TX','snap_WI', 'dept_id', 'cat_id', 'store_id', 'state_id'])

In [7]:
# create function to change granularity: aggregate items by store and state to deal with larger sales values
def macro_granularity(df):
    df_same = df.groupby(['item_id','date'])[['day','week', 'month', 'year', 'dayofweek', 'weekend', 'dayofyear',
       'date_block_num_month', 'date_block_num_week', 'date_block_num_day', 'OrthodoxChristmas', 'OrthodoxEaster',
       'LaborDay', 'ValentinesDay', 'PresidentsDay', 'MartinLutherKingDay',
       'LentWeek2', 'Christmas', 'NBAFinalsEnd', 'Chanukah End', 'VeteransDay',
       'IndependenceDay', 'LentStart', 'Pesach End', "Mother's day",
       'NBAFinalsStart', 'MemorialDay', 'Thanksgiving', 'Eid al-Fitr',
       "Father's day", 'Halloween', 'EidAlAdha', 'Easter', 'StPatricksDay',
       'ColumbusDay', 'Ramadan starts', 'Purim End', 'Cinco De Mayo',
       'SuperBowl', 'NewYear']].agg('max').reset_index()
    df_same = df_same.drop(columns=['item_id','date'])
    df_sum = df.groupby(['item_id','date'])[['sales',
       'release', 'sales_lag_days_8',
       'sales_lag_days_9', 'sales_lag_days_10', 'sales_lag_days_11',
       'sales_lag_days_12', 'sales_lag_days_13', 'sales_lag_days_14',
       'sales_lag_days_15', 'sales_lag_days_16', 'sales_lag_days_17',
       'sales_lag_days_18', 'sales_lag_days_19', 'sales_lag_days_20',
       'sales_lag_days_21', 'sales_lag_days_22', 'sales_lag_days_23',
       'sales_lag_days_24', 'sales_lag_days_25', 'sales_lag_days_26',
       'sales_lag_days_27', 'sales_lag_days_28', 'sales_lag_days_29',
       'sales_lag_days_30', 'sales_lag_days_31', 'sales_lag_days_32',
       'sales_lag_days_33', 'sales_lag_days_34', 'sales_lag_days_35',
       'sales_lag_days_365', 'sales_lag_days_366']].agg(['sum']).reset_index()
    df_mean = df.groupby(['item_id','date'])[['sell_price','cum_mean_week_item_id', 'cum_mean_day_item_id',
       'cum_mean_month_item_id', 'diff_price_same_month_item',
       'diff_price_same_day_item', 'diff_price_same_item',
       'min_price_same_store_item', 'max_price_same_store_item',
       'mean_price_same_store_item', 'std_price_same_store_item',
       'nunique_price_same_store_item', 'price_norm', 'price_momentum',
       'price_momentum_m', 'price_momentum_y','mean_8-14_days_sales',
       'mean_8-21_days_sales', 'mean_8-28_days_sales', 'mean_8-35_days_sales',
       'std_8-14_days_sales', 'std_8-35_days_sales', 'mean_15-21_days_sales',
       'mean_15-28_days_sales', 'mean_15-35_days_sales',
       'mean_15-42_days_sales', 'std_15-21_days_sales', 'std_15-42_days_sales',
       'mean_22-28_days_sales', 'mean_22-35_days_sales',
       'mean_22-42_days_sales', 'mean_22-49_days_sales',
       'std_22-28_days_sales', 'std_22-49_days_sales', 'mean_29-35_days_sales',
       'mean_29-42_days_sales', 'mean_29-49_days_sales',
       'mean_29-56_days_sales', 'std_29-35_days_sales', 'std_29-56_days_sales',
       'date_avg_sales_lag_days_1', 'date_avg_sales_lag_days_8',
       'date_avg_sales_lag_days_15', 'date_avg_sales_lag_days_22',
       'date_avg_sales_lag_days_29', 'date_avg_sales_lag_days_100',
       'date_avg_sales_lag_days_365', 'date_store_avg_sales_lag_days_1',
       'date_store_avg_sales_lag_days_8','date_store_avg_sales_lag_days_15', 'date_store_avg_sales_lag_days_22',
       'date_store_avg_sales_lag_days_29', 'date_store_avg_sales_lag_days_100',
       'date_store_avg_sales_lag_days_365', 'date_item_avg_sales_lag_days_1',
       'date_item_avg_sales_lag_days_8', 'date_item_avg_sales_lag_days_15',
       'date_item_avg_sales_lag_days_22', 'date_item_avg_sales_lag_days_29',
       'date_item_avg_sales_lag_days_100', 'date_item_avg_sales_lag_days_365', 'item_id_nanmean_encod', 'item_id_nanstd_encod',
       'item_id_cum_mean_encod']].agg(['mean']).reset_index()
    df_mean = df_mean.drop(columns=['item_id','date'])
    df_final = pd.concat([df_sum,df_mean,df_same.reset_index()],axis=1)
    return df_final, df_same, df_sum, df_mean

In [9]:
df_macro_2 = macro_granularity(df_macro)

In [10]:
df_new = df_macro_2[0]

In [11]:
df_new.head()

,"(item_id, )","(date, )","(sales, sum)","(release, sum)","(sales_lag_days_8, sum)","(sales_lag_days_9, sum)","(sales_lag_days_10, sum)","(sales_lag_days_11, sum)","(sales_lag_days_12, sum)","(sales_lag_days_13, sum)",...,Halloween,EidAlAdha,Easter,StPatricksDay,ColumbusDay,Ramadan starts,Purim End,Cinco De Mayo,SuperBowl,NewYear
0,FOODS_1_045,2012-01-31,15.0,8,16,17,26,34,13,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.0,NaN
1,FOODS_1_045,2012-02-01,10.0,8,17,16,17,26,34,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.0,NaN
2,FOODS_1_045,2012-02-02,9.0,8,15,17,16,17,26,34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.0,NaN
3,FOODS_1_045,2012-02-03,17.0,8,9,15,17,16,17,26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.0,NaN
4,FOODS_1_045,2012-02-04,23.0,8,14,9,15,17,16,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN


In [12]:
df_old = df_final

In [15]:
df_new.shape[0]

480600

### EDA: old vs new dataset

In [16]:
#percentage of zeros
#old
sales_old = df_old['sales']
print('Percentage of zeros in sales with original data', sales_old[sales_old == 0].count()/df_old.shape[0])

#new 
sales_new = df_new.iloc[:,2]
print('Percentage of zeros in sales with compressed data', sales_new[sales_new == 0].count()/df_new.shape[0])

Percentage of zeros in sales with original data 0.5859311630683854
Percentage of zeros in sales with compressed data 0.26446525176862257


In [17]:
#rename columns
def rename_columns(df_new):
    nb_col = df_new.columns.shape[0]
    col_names = [0] * nb_col
    for n in range(nb_col):
        if len(df_new.columns[n])==2:
            col_names[n] = df_new.columns[n][0]
        else: col_names[n] = df_new.columns[n]
    df_new.columns = col_names
    return df_new
    

In [18]:
df_new = rename_columns(df_new)

In [19]:
df_new.head()

,item_id,date,sales,release,sales_lag_days_8,sales_lag_days_9,sales_lag_days_10,sales_lag_days_11,sales_lag_days_12,sales_lag_days_13,...,Halloween,EidAlAdha,Easter,StPatricksDay,ColumbusDay,Ramadan starts,Purim End,Cinco De Mayo,SuperBowl,NewYear
0,FOODS_1_045,2012-01-31,15.0,8,16,17,26,34,13,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.0,NaN
1,FOODS_1_045,2012-02-01,10.0,8,17,16,17,26,34,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.0,NaN
2,FOODS_1_045,2012-02-02,9.0,8,15,17,16,17,26,34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.0,NaN
3,FOODS_1_045,2012-02-03,17.0,8,9,15,17,16,17,26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.0,NaN
4,FOODS_1_045,2012-02-04,23.0,8,14,9,15,17,16,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN


In [ ]:
df_new = df_new.drop(columns='index')

### Saving prepared data

Finally, the dataframe with all features is split by stores and saved, as models will be specific to each store.

In [20]:
pd.to_pickle(df_old, "M5_df" + ".pkl")

In [21]:
pd.to_pickle(df_new, "M5_df_macro" + ".pkl")